# CPM Method of 2D Keypoint Estimation

## Imports and Setup

In [ ]:
import os
import src.bug_dataset
from src.models.cpm_model import CPM
from src.train_cpm_network import Train_CPM_Network
import numpy as np
import pandas as pd
from skimage import io
import torch
from torch.utils.data import DataLoader
from torchvision import transforms
import matplotlib.pyplot as plt
plt.ion()   # interactive mode

In [ ]:
target_dir = "../data/single_sungaya/"
out_df = pd.read_hdf(os.path.join(target_dir, "Data_3D_Pose.hdf5"))

In [ ]:
reduceKeypoints = True
sungaya_dataset = src.bug_dataset.BugDataset(df=out_df,reduced=reduceKeypoints,
                             root_dir=target_dir,transform=transforms.Compose([
                                src.bug_dataset.ToTensor()
                                   ]))

In [ ]:
import cv2
sample = sungaya_dataset[2417]
name = sample['file_name']
image = sample['image']
img=image.numpy()
img=np.swapaxes(img,0,1)
img=np.swapaxes(img,1,2)

name = sample['file_name']
keypoints = sample['key_points_2D']

keypoints = (sample['key_points_2D']*sungaya_dataset.std_2d)+sungaya_dataset.means_2d

heatmap = sample['heatmap'].numpy()

fig = plt.figure()
ax = fig.add_subplot()

ax.imshow(img.astype('uint8'))
heat =ax.imshow(cv2.resize(heatmap[0], (152,152)), alpha = 0.3,cmap='hot')
# ax.scatter(keypoints[:,0], keypoints[:,1], marker='o', s=5,  color='b')

ax.set_xlim([0,152])
ax.set_ylim([152,0])
ax.set_aspect('equal')

plt.title(name)
fig.colorbar(heat)
plt.show()

In [ ]:
train_split = 0.7
valid_split = 0.1
train_size = int(train_split * len(sungaya_dataset))
valid_size = int(valid_split * len(sungaya_dataset))
test_size = len(sungaya_dataset) - (train_size+valid_size)
train_dataset,valid_dataset ,test_dataset = torch.utils.data.random_split(sungaya_dataset, [train_size, valid_size, test_size], generator=torch.Generator().manual_seed(42))

In [ ]:
batch_size = 16

train_dataloader = DataLoader(train_dataset, batch_size=batch_size)
valid_dataloader = DataLoader(valid_dataset, batch_size=batch_size)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

In [ ]:
# Create Model
if reduceKeypoints:
    model = CPM(28)
    accurate_dist = torch.from_numpy(np.array([[10,10]]*28))
else:
    model = CPM(62)
    accurate_dist = torch.from_numpy(np.array([[10,10]]*62))

In [ ]:
load = False
if load == True:
    path = "..\model\estimator2d.pt"
    model.load_state_dict(torch.load(path))
    model.eval()

epochs = 1500
learning_rate = 2e-6  #-6
optimizer = torch.optim.Adam(params=model.parameters(), lr=learning_rate, betas=(0.5, 0.999))
liftingtrainer = Train_CPM_Network(model,optimizer, accurate_dist, train_dataloader, valid_dataloader)
trained_model = liftingtrainer.run(epochs)

In [ ]:
trained_model = liftingtrainer.model
trained_model.eval()
# Code get the first batch of results
with torch.no_grad():
    for data in train_dataloader:
        image = data['image']
        center = data['centermap']
        heatmap = data['heatmap']
        file_name = data['file_name']
        kp = data['key_points_2D']
        visib = data['visibility']
        input_var = image.to(liftingtrainer.device, dtype=torch.float)
        heatmap_var = heatmap.to(liftingtrainer.device, dtype=torch.float)
        centermap_var = center.to(liftingtrainer.device, dtype=torch.float)

        heat1, heat2, heat3, heat4, heat5, heat6 = trained_model(input_var, centermap_var)
        break

In [ ]:
def get_kpt(maps, img_h = 152.0, img_w = 152.0):
        # maps (1,63,76,76)
        maps = maps.clone().cpu().data.numpy()
        map_6 = maps

        kpts = []
        for m in map_6[1:]:
            h, w = np.unravel_index(m.argmax(), m.shape)
            x = int(w * img_w / m.shape[1])
            y = int(h * img_h / m.shape[0])
            kpts.append([x,y])
        return np.array(kpts)

In [ ]:
sample = 7
limb = 0
name = file_name[sample]

img = image[sample].numpy()
img=np.swapaxes(img,0,1)
img=np.swapaxes(img,1,2)

pred = heat6[sample].cpu().numpy()
pred_kp = get_kpt(heat6[sample])

acc = heatmap[sample].cpu().numpy()
acc_kp = get_kpt(heatmap[sample])

vis = visib[sample].cpu().numpy()
keypoints = (kp[sample].numpy()*sungaya_dataset.std_2d)+sungaya_dataset.means_2d

plt.figure()
plt.imshow(img)

plt.scatter(pred_kp[limb,0]*vis[limb], pred_kp[limb,1]*vis[limb], marker='o', s=10,  color='r')
plt.scatter(acc_kp[limb,0]*vis[limb], acc_kp[limb,1]*vis[limb], marker='o', s=10,  color='b')

plt.imshow(cv2.resize(pred[limb+1], (152,152)), alpha = .5,cmap='hot')
# plt.imshow(cv2.resize(acc[limb+1], (152,152)), alpha = .5,cmap='hot')

plt.colorbar()
plt.title(name)
plt.show()
# ((abs(pred_kp- acc_kp)<np.array([1,1])).sum().item())/(28*2)

In [ ]:
# https://pytorch.org/tutorials/beginner/saving_loading_models.html
# PATH = "..\models\estimator2d_with_heatmap_mse.pt"
# torch.save(trained_model.state_dict(), PATH)

In [ ]:
# Display recent training loss graph
file_path = '..\\results\cpm_model_20220308-004023_train_loss.txt'
losses = []
with open(file_path) as file:
    losses = [float(line.rstrip()) for line in file]

epochs = list(range(1,len(losses)+1))

In [ ]:
plt.figure() 
plt.plot(epochs,losses)
plt.show()
